# 2.11 Matrix free operator application
Usually, we assemble matrices in sparse matrix format. Certain methods allow to improve performance and reduce memory requirements considereably by avoiding building and storing the system matrix. The counterpart of this approach is that it is now difficult to build preconditioners

# Hybrid mixed method
We 

In [ ]:
import netgen.gui
from netgen.geom2d import unit_square
from ngsolve import *

mesh = Mesh(unit_square.GenerateMesh(maxh=0.3))
order=1
Sigma = VectorL2(mesh, order=order, piola=True)
Vt = L2(mesh, order=order-1)
Vf = FacetFESpace(mesh, order=order)
V = FESpace([Vt,Vf])
sigma,tau = Sigma.TnT()
(ut,uf), (vt,vf) = V.TnT()

In [ ]:
a = BilinearForm(Sigma)
b = BilinearForm(trialspace=Sigma, testspace=V)
b += div(sigma) * vt * dx
n = specialcf.normal(mesh.dim)
dS = dx(element_boundary=True)
b += sigma*n*vf * dS
b.Assemble()

In [ ]:
for el in mesh.Elements():
    felS = Sigma.GetFE(el)
    felV = V.GetFE(el)
    fel = MixedFE(felS, felV)
    trafo = mesh.GetTrafo(el)
    
    # try integratos 0 and 1 ...
    elmat = b.integrators[0].CalcElementMatrix(fel, trafo)   
    print (elmat)

## geometry-free matrix multiplication 


In [ ]:
from netgen.csg import *
geom = CSGeometry()
geom.Add (Sphere(Pnt(50,50,50),80) \
          -Cylinder(Pnt(-100,0,0),Pnt(200,0,0), 40) \
          -Cylinder(Pnt(100,-100,100),Pnt(100,200,100),40)
          -Cylinder(Pnt(0,100,-100), Pnt(0,100,200),40)
          -Sphere(Pnt(50,50,50),50))
# geom.Draw()

mesh = Mesh(geom.GenerateMesh(maxh=25))
mesh.Curve(5)
# Draw (mesh)

In [ ]:
order=5
Sigma = VectorL2(mesh, order=order, piola=True)
Vt = L2(mesh, order=order-1)
Vf = FacetFESpace(mesh, order=order, dirichlet=[2])
V = FESpace([Vt,Vf])
print ("Sigma.ndof =", Sigma.ndof, ", V.ndof =", V.ndof)
sigma,tau = Sigma.TnT()
(ut,uf), (vt,vf) = V.TnT()

b = BilinearForm(trialspace=Sigma, testspace=V, geom_free=True)
b += div(sigma) * vt * dx
n = specialcf.normal(mesh.dim)
dS = dx(element_boundary=True)
b += sigma*n*vf * dS
b.Assemble()

In [ ]:
x = b.mat.CreateRowVector()
y = b.mat.CreateColVector()
x[:] = 1

%timeit y.data = b.mat * x


In [ ]:
ainv = Sigma.Mass(rho=1).Inverse()
Laplace = b.mat @ ainv @ b.mat.T

y2 = ainv.CreateColVector()
%timeit y2.data = ainv * x

In [ ]:
f = LinearForm (V)
f += -1*vt * dx
f.Assemble()

proj = Projector(V.FreeDofs(), True)

gfu = GridFunction (V)
from time import time
start = time()
with TaskManager():
  solvers.CG(mat=Laplace, pre=proj, rhs=f.vec, sol=gfu.vec, maxsteps=5000, printrates=False)
print ("time =", time()-start)

In [ ]:
Draw (gfu.components[1])

In [ ]:
help (solvers.CG)